In [ ]:
!pip install -q pyngrok

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install -q streamlit-ace

In [ ]:
!pip install -q fuzzywuzzy[speedup]

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import re
from fuzzywuzzy import process
import json
import requests
import pandas as pd



def main():

  namesData = requests.get('https://cspinheiro.github.io/interpol.json')

  df = pd.read_csv('Endpoints.csv')
  apiNameOptions = df['N'].drop([0,0])
  apiRoutesOptions = df['Endpoint'].drop([0, 0])
  apiRoutesOptions = apiRoutesOptions.values.tolist()
  apiNameOptions = apiNameOptions.values.tolist()

  apiRoutesOptions[12] = 'https://rodrigofer89.github.io/npl.json'

  endpointOptions = {apiNameOptions[i]: apiRoutesOptions[i] for i in range(len(apiNameOptions))}

  def handleAirportSelection(airport: str, allAirports : bool):
    if allAirports:
      return getAllEndpointsData()
    
    route = endpointOptions[airport.upper()]
    response = requests.get(route).json()

    totalData = []
    for i in response:
      for data in response[i]:
        totalData.append(*data.values())

    return totalData

  def getAllEndpointsData():
    
    totalData = []

    for endpoint in endpointOptions.values():
      response = requests.get(endpoint).json()
      for option in response:
        for data in response[option]:
          totalData.append(*data.values())

    return totalData

  def handleBlockedPassports(passportNumber, allAirport):

    passportLocation = re.findall('[a-zA-Z]+', passportNumber)
    passport = re.findall('[0-9]+', passportNumber)

    if not passportLocation:
      allAirport = True
      passportLocation = ''
    else :
      passportLocation = str(passportLocation[0])
    
    if not passport:
      st.warning("Por favor informe o número do seu passaporte!")
      return

    blockedData = handleAirportSelection(passportLocation, allAirport)

    if int(passport[0]) in blockedData:
      return 'Proibido'
    
    return 'Permitido'

  def handleBlockedPersons(name):

    blockedNamesData = namesData.json()['interpol']

    blocked = []

    for data in blockedNamesData:
      for i, v in data.items():
        blocked.append(v)

    searchList = process.extract(str(name), blocked)

    for i in searchList:
      if i[1] >= 72:
        return 'Proibido'

      return 'Permitido'


  def handleImigrate(name, passportNumber, allAirport):

      passport = handleBlockedPassports(passportNumber, allAirport)

      if not passport:
        return

      person = handleBlockedPersons(name)

      if passport == 'Proibido' or person == 'Proibido':
         st.error("Você não está permitido entrar nesse País")
         return
      
      st.success("Você pode entrar nesse País")
      return

  st.set_page_config(page_title="Aeroporto", page_icon="✈️", layout="centered")
  st.title("📋 Segurança do Aeroporto")
  form = st.form(key="form")

  apiNameOptions.append('ALL')

  with form:
    name = st.text_input("Nome Completo:")
    passport = st.text_input("Número do Passaporte:")
    allAirport = st.checkbox("Todos os Aeroportos")
    submitted = st.form_submit_button(label="Verificar")

  if submitted:
    if not name or not passport:
      st.warning("Preencha todos os Campos!")
    else :
      handleImigrate(name, passport, allAirport)

if __name__ == '__main__':
  main()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501